# **Fraud Detection Using Variational AutoEncoders**





**This is the list of our group members:**

1. Vadapalli Sai Sravan (CS24MTECH02007)
2. Supreet Shukla (CS24MTECH02004)
3. Tarun Jangir (CS24MTECH02005)
4. Taufique Ramzan Shaikh (CS24MTECH02006)
5. Afzaal Ahmad (CS24MTECH02002)

# **Training the First Variational Auto Encoder**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load and preprocess data
data = pd.read_csv("/content/drive/MyDrive/creditcard.csv")

# Separate features and labels
X = data.drop('Class', axis=1)
y = data['Class']

# Log-scale the time difference and amount features
time_diff_idx = X.columns.get_loc('Time')
amount_idx = X.columns.get_loc('Amount')

X.iloc[:, time_diff_idx] = np.log1p(X.iloc[:, time_diff_idx])  # Log-scale time difference
X.iloc[:, amount_idx] = np.log1p(X.iloc[:, amount_idx])  # Log-scale amount

# Normalize features to the range (0, 1)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Separate fraud and non-fraud transactions
non_fraud_idx = np.where(y == 0)[0]
fraud_idx = np.where(y == 1)[0]

# Split non-fraud data into train and validation sets
X_non_fraud = X[non_fraud_idx]
y_non_fraud = y[non_fraud_idx]
X_train, X_val, y_train, y_val = train_test_split(X_non_fraud, y_non_fraud, test_size=0.001, random_state=42)

# Create test set with remaining non-fraud data and all fraud data
X_test_non_fraud = np.delete(X[non_fraud_idx], np.concatenate((np.arange(len(X_train)), np.arange(len(X_val)))), axis=0)
X_test = np.concatenate((X_test_non_fraud, X[fraud_idx]), axis=0)
y_test_non_fraud = np.delete(y[non_fraud_idx], np.concatenate((np.arange(len(y_train)), np.arange(len(y_val)))), axis=0)
y_test = np.concatenate((y_test_non_fraud, y[fraud_idx]), axis=0)

# Define the variational autoencoder architecture
input_dim = X_train.shape[1]
latent_dim = 14

# Encoder architecture
encoder_input = Input(shape=(input_dim,))
x = Dense(64, activation='relu')(encoder_input)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder architecture
decoder_input = Input(shape=(latent_dim,))
x = Dense(64, activation='relu')(decoder_input)
decoder_output = Dense(input_dim, activation='sigmoid')(x)

# Define the VAE model
encoder = Model(encoder_input, [z_mean, z_log_var, z], name='encoder')
decoder = Model(decoder_input, decoder_output, name='decoder')
vae_output = decoder(encoder(encoder_input)[2])
vae = Model(encoder_input, vae_output, name='vae')

# Define the VAE loss
reconstruction_loss = mse(encoder_input, vae_output)
kl_loss = -0.5 * tf.keras.backend.mean(1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var), axis=-1)
vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

# Compile the VAE model
vae.compile(optimizer='adam')

# Train the VAE
vae.fit(X_train, X_train, epochs=100, batch_size=32, shuffle=True, validation_data=(X_val, X_val))

# Compute reconstruction errors for training data
train_reconstructions = vae.predict(X_train)
reconstruction_errors_train = np.mean(np.square(X_train - train_reconstructions), axis=1)

# Compute the mean reconstruction error on non-fraud training data
non_fraud_train_errors = reconstruction_errors_train

# Calculate threshold using the reconstruction errors of non-fraudulent transactions in the training set
threshold = np.mean(non_fraud_train_errors) + 1.5 * np.std(non_fraud_train_errors)

# Compute reconstruction errors for testing data
test_reconstructions = vae.predict(X_test)
reconstruction_errors_test = np.mean(np.square(X_test - test_reconstructions), axis=1)

# Classify transactions as fraud or non-fraud based on the threshold
y_pred_test = [1 if e > threshold else 0 for e in reconstruction_errors_test]

Epoch 1/100
8876/8876 [==============================] - 23s 2ms/step - loss: 0.0031 - val_loss: 0.0026
Epoch 2/100
8876/8876 [==============================] - 24s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 3/100
8876/8876 [==============================] - 28s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 4/100
8876/8876 [==============================] - 21s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 5/100
8876/8876 [==============================] - 22s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 6/100
8876/8876 [==============================] - 20s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 7/100
8876/8876 [==============================] - 22s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 8/100
8876/8876 [==============================] - 20s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 9/100
8876/8876 [==============================] - 22s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 10/100
8876/8876 [==============================] - 21s 2m

# **Evaluate model performance on test set using first Variational Auto Encoder**

In [ ]:
from sklearn.metrics import precision_score,  recall_score

precision = precision_score(y_test, y_pred_test)
print(f"precision is: {precision}")
recall = recall_score(y_test, y_pred_test)
print(f"recall is: {recall}")

# Evaluate model performance on test set
f1 = f1_score(y_test, y_pred_test)
print(f'F1 score on test set: {f1}')

precision is: 0.9691211401425178
recall is: 0.8292682926829268
F1 score on test set: 0.8937568455640744


# **Training the Second Variational Auto Encoder**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load and preprocess data
data = pd.read_csv("/content/drive/MyDrive/creditcard.csv")

# Separate features and labels
X = data.drop('Class', axis=1)
y = data['Class']

# Log-scale the time difference and amount features
time_diff_idx = X.columns.get_loc('Time')
amount_idx = X.columns.get_loc('Amount')

X.iloc[:, time_diff_idx] = np.log1p(X.iloc[:, time_diff_idx])  # Log-scale time difference
X.iloc[:, amount_idx] = np.log1p(X.iloc[:, amount_idx])  # Log-scale amount

# Normalize features to the range (0, 1)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Separate fraud and non-fraud transactions
non_fraud_idx = np.where(y == 0)[0]
fraud_idx = np.where(y == 1)[0]

# Split non-fraud data into train and validation sets
X_non_fraud = X[non_fraud_idx]
y_non_fraud = y[non_fraud_idx]
X_train, X_val, y_train, y_val = train_test_split(X_non_fraud, y_non_fraud, test_size=0.001, random_state=42)

# Create test set with remaining non-fraud data and all fraud data
X_test_non_fraud = np.delete(X[non_fraud_idx], np.concatenate((np.arange(len(X_train)), np.arange(len(X_val)))), axis=0)
X_test = np.concatenate((X_test_non_fraud, X[fraud_idx]), axis=0)
y_test_non_fraud = np.delete(y[non_fraud_idx], np.concatenate((np.arange(len(y_train)), np.arange(len(y_val)))), axis=0)
y_test = np.concatenate((y_test_non_fraud, y[fraud_idx]), axis=0)

# Define the variational autoencoder architecture
input_dim = X_train.shape[1]
latent_dim = 12  # Choose the dimension of the latent space

# Encoder architecture
encoder_input = Input(shape=(input_dim,))
x = Dense(20, activation='relu')(encoder_input)  # Reduced size hidden layer
x = Dense(12, activation='relu')(x)  # Additional hidden layer
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder architecture
decoder_input = Input(shape=(latent_dim,))
x = Dense(12, activation='relu')(decoder_input)  # Additional hidden layer
x = Dense(20, activation='relu')(x)  # Reduced size hidden layer
decoder_output = Dense(input_dim, activation='sigmoid')(x)

# Define the VAE model
encoder = Model(encoder_input, [z_mean, z_log_var, z], name='encoder')
decoder = Model(decoder_input, decoder_output, name='decoder')
vae_output = decoder(encoder(encoder_input)[2])
vae = Model(encoder_input, vae_output, name='vae')

# Define the VAE loss
reconstruction_loss = mse(encoder_input, vae_output)
kl_loss = -0.5 * tf.keras.backend.mean(1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var), axis=-1)
vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

# Compile the VAE model
vae.compile(optimizer='adam')

# Train the VAE
vae.fit(X_train, X_train, epochs=100, batch_size=32, shuffle=True, validation_data=(X_val, X_val))

# Compute reconstruction errors for training data
train_reconstructions = vae.predict(X_train)
reconstruction_errors_train = np.mean(np.square(X_train - train_reconstructions), axis=1)

# Compute the mean reconstruction error on non-fraud training data
non_fraud_train_errors = reconstruction_errors_train

# Calculate threshold using the reconstruction errors of non-fraudulent transactions in the training set
threshold = np.mean(non_fraud_train_errors) + 1.5 * np.std(non_fraud_train_errors)  # Adjust threshold as needed

# Compute reconstruction errors for testing data
test_reconstructions = vae.predict(X_test)
reconstruction_errors_test = np.mean(np.square(X_test - test_reconstructions), axis=1)

# Classify transactions as fraud or non-fraud based on the threshold
y_pred_test = [1 if e > threshold else 0 for e in reconstruction_errors_test]


Epoch 1/100
8876/8876 [==============================] - 23s 2ms/step - loss: 0.0031 - val_loss: 0.0026
Epoch 2/100
8876/8876 [==============================] - 22s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 3/100
8876/8876 [==============================] - 21s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 4/100
8876/8876 [==============================] - 23s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 5/100
8876/8876 [==============================] - 22s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 6/100
8876/8876 [==============================] - 23s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 7/100
8876/8876 [==============================] - 23s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 8/100
8876/8876 [==============================] - 24s 3ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 9/100
8876/8876 [==============================] - 25s 3ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 10/100
8876/8876 [==============================] - 23s 3m

# **Evaluate model performance on test set using second Variational Auto Encoder**

In [ ]:
# Evaluate model performance on test set
from sklearn.metrics import precision_score,  recall_score

precision = precision_score(y_test, y_pred_test)
print(f"precision is: {precision}")
recall = recall_score(y_test, y_pred_test)
print(f"recall is: {recall}")

f1 = f1_score(y_test, y_pred_test)
print(f'F1 score on test set: {f1}')

precision is: 0.9691211401425178
recall is: 0.8292682926829268
F1 score on test set: 0.8937568455640744


# **SUMMARY**

**Introduction**: This code implements a Variational Autoencoder (VAE) for anomaly detection in credit card transactions. The VAE is a deep generative model that learns to reconstruct the input data while also learning a compressed, latent representation of the data.

**Data Preprocessing**: The code loads a credit card transaction dataset from a CSV file and preprocesses it. The target variable 'Class' (fraud or non-fraud) is separated from the features. The 'Time' and 'Amount' features are log-scaled, and all features are normalized to the range (0, 1) using MinMaxScaler. The dataset is then split into training, validation, and test sets, ensuring that only non-fraudulent transactions are included in the train set and ensuring that the test set is a combination of fraud and non-fraud transactions.

**VAE Architecture**: The VAE consists of an encoder and a decoder network. The encoder maps the input data to a lower-dimensional latent space, capturing the essential features of the data. The decoder reconstructs the original data from the latent space representation. The architectures of the encoder and decoder are defined using dense layers with ReLU activations.

**Model Training**: The VAE is trained on the non-fraudulent transactions in the training set. The loss function is a combination of the reconstruction loss (mean squared error) and the Kullback-Leibler (KL) divergence loss, which regularizes the latent space. The model is trained for 100 epochs, and the validation set is used to monitor the training process. We trained two auto encoders: First autoencoder: 1 encoder layer, 1 bottleneck layer, 1 decoder layer Second autoencoder: 2 encoder layers, 1 bottleneck layer, 2 decoder layers

**Anomaly Detection**: After training, the VAE computes the reconstruction errors for the training and test sets. The reconstruction error is the mean squared difference between the original input and its reconstructed counterpart. A threshold is calculated based on the reconstruction errors of non-fraudulent transactions in the training set. Transactions with reconstruction errors above the threshold are classified as anomalies (potential fraud).

**Model Evaluation**: The code evaluates the VAE's performance on the test set using the F1-score, precision, and recall metrics for the anomaly detection task.The code also includes a variation of the VAE architecture, where the number of hidden layers and their sizes are modified to explore the impact on model performance.

**Performance metrics of First VAE:**

precision is: 0.9691211401425178

recall is: 0.8292682926829268

F1 score on test set: 0.8937568455640744

**Performance metrics of Second VAE:**

precision is: 0.9691211401425178

recall is: 0.8292682926829268

F1 score on test set: 0.8937568455640744